# ANLI Baseline with LLM

You have to implement in this notebook a baseline for ANLI classification using an LLM.
This baseline must be implemented using DSPy.



In [ ]:
# Configure the DSPy environment with the language model - for grok the parameters must be:
# env variable should be in os.environ['XAI_API_KEY']
# "xai/grok-3-mini"
import os
import dspy

# lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
# for ollama
lm = dspy.LM('ollama_chat/llama3.1', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [ ]:
from typing import Literal

## Implement the DSPy classifier program.
class NLIClassifier(dspy.Signature):
    premise     :str = dspy.InputField(desc = "The premise")
    hypothesis  :str = dspy.InputField(desc="The hypothesis based on the premise")
    label       : Literal["entailment", "neutral", "contradiction"] = dspy.OutputField(desc="Return exactly one of: "
                        "'entailment', 'neutral', 'contradiction'")

predictor = dspy.Predict(NLIClassifier,llm=lm)

def zero_shot_nli_classifier(x):
    # if hasattr(x,'premise') and hasattr(x,'hypothesis'):
    return predictor(premise=x['premise'], hypothesis=x['hypothesis']).label
    # print("ERROR")

## Load ANLI dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("facebook/anli")
dataset = dataset.filter(lambda x: x['reason'] != None and x['reason'] != "")

In [ ]:
dataset

In [16]:
example = dataset['test_r3'][0]
print(example)
print(zero_shot_nli_classifier(example))

{'uid': 'b0e63408-53af-4b46-b33d-bf5ba302949f', 'premise': "It is Sunday today, let's take a look at the most popular posts of the last couple of days. Most of the articles this week deal with the iPhone, its future version called the iPhone 8 or iPhone Edition, and new builds of iOS and macOS. There are also some posts that deal with the iPhone rival called the Galaxy S8 and some other interesting stories. The list of the most interesting articles is available below. Stay tuned for more rumors and don't forget to follow us on Twitter.", 'hypothesis': 'The day of the passage is usually when Christians praise the lord together', 'label': 0, 'reason': "Sunday is considered Lord's Day"}


2025/07/09 17:34:04 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


RuntimeError: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.APIConnectionError: Ollama_chatException - Object of type LM is not JSON serializable

## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [ ]:
from evaluate import load

accuracy = load("accuracy")
precision = load("precision")
recall = load("recall")
f1 = load("f1")


In [ ]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [ ]:
clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])

## Your Turn

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]